In [44]:
import pandas as pd 
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

In [45]:
from config import DB_USER, DB_PASSWORD, DB_SERVER_NAME, DB_DATABASE_NAME
connection_url = URL.create(
    drivername = "postgresql+pg8000", 
    username = DB_USER,
    password = DB_PASSWORD,
    host = DB_SERVER_NAME, 
    port = 5432,
    database = DB_DATABASE_NAME, 
)

engine = create_engine(connection_url)

In [46]:
atmosphere_df = pd.read_sql("atmosphere", engine)
city_df = pd.read_sql("city", engine).rename(columns={"name": "city_name"})
temperature_df = pd.read_sql("temperature", engine)

In [47]:
staging_df = pd.merge(atmosphere_df, temperature_df, on=["city_id", "datetime"], how="inner")
staging_df["date"] = staging_df["datetime"].astype('datetime64').dt.date
merged_df = pd.merge(staging_df, city_df, on=["city_id"], how="inner")
merged_df["city_name"] = merged_df["city_name"].str.lower()

In [48]:
merged_df.head()

,city_id,datetime,main_pressure,main_humidity,main_temp,main_feels_like,main_temp_min,main_temp_max,date,city_name,coord_lon,coord_lat
0,2172517,2021-12-01 14:03:41,1016.0,81.0,288.45,288.15,287.14,289.68,2021-12-01,canberra,149.1281,-35.2835
1,2172517,2021-12-01 14:26:29,1016.0,82.0,288.15,287.85,287.07,289.12,2021-12-01,canberra,149.1281,-35.2835
2,2172517,2021-12-01 14:50:19,1016.0,82.0,288.08,287.77,286.51,289.12,2021-12-01,canberra,149.1281,-35.2835
3,2172517,2021-12-01 16:18:30,1016.0,87.0,13.53,13.21,11.99,14.86,2021-12-01,canberra,149.1281,-35.2835
4,2172517,2021-12-01 15:53:32,1016.0,86.0,287.10,286.80,285.96,288.56,2021-12-01,canberra,149.1281,-35.2835


In [49]:
grouped_df = merged_df[[
    "main_pressure", 
    "main_humidity", 
    "city_name",
    "date",
    "main_temp"
]].groupby(by=["city_name", "date"]).mean().reset_index()
grouped_df = grouped_df[grouped_df["main_temp"]<50] # remove temps that were recorded in farenheit 
grouped_df.head()

,city_name,date,main_pressure,main_humidity,main_temp
1,adelaide,2021-12-02,1019.552632,69.736842,14.189211
2,adelaide,2021-12-03,1020.069444,59.291667,18.010000
3,adelaide,2021-12-04,1020.542169,54.831325,19.109277
4,adelaide,2021-12-05,1012.875000,55.000000,19.069375
5,adelaide,2021-12-06,1011.875000,53.227273,21.157045


In [50]:
clean_df = pd.get_dummies(grouped_df,columns=["city_name"])

In [51]:
X = clean_df[[
    "main_pressure", 
    "main_humidity", 
    "city_name_adelaide", 
    "city_name_brisbane", 
    "city_name_canberra", 
    "city_name_darwin", 
    "city_name_hobart", 
    "city_name_melbourne", 
    "city_name_perth", 
    "city_name_sydney"
]]

y = clean_df["main_temp"].values.reshape(-1,1)

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [53]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

/Users/jonathanneo/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [54]:
# scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

/Users/jonathanneo/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  FutureWarning,
/Users/jonathanneo/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [55]:
# train the model 
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_scaled, y_train_scaled)
model.score(X_test_scaled, y_test_scaled)

0.8802114045348468

In [56]:
# create a city encoder 
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
encoder.fit(grouped_df[[
    "city_name"
]])

OneHotEncoder()

In [57]:
# create a prediction function 
def predict(model, encoder, X_scaler, y_scaler, user_inputs):
    # get the user input data 
    pressure = user_inputs["pressure"]
    humidity = user_inputs["humidity"]
    city_name = user_inputs["city_name"]
    
    # store city names into a df 
    city_input_df = pd.DataFrame({
        "city_name": [city_name]
    })

    # use encoder to transform the city df 
    X_transformed = encoder.transform(city_input_df)
    city_df = pd.DataFrame(columns=[*encoder.categories_], data=X_transformed.toarray())
    
    # store pressure and humidty into df 
    input_df = pd.DataFrame({
        "pressure": [pressure],
        "humidity": [humidity]
    })

    # combine both df's using indexes 
    df = input_df.merge(city_df, left_index=True, right_index=True)

    # scale the X input df 
    X_scaled = X_scaler.transform(df)

    # obtain prediction (y) 
    prediction_scaled = model.predict(X_scaled)
    
    # scale prediction to human readable terms i.e. celcius 
    prediction = y_scaler.inverse_transform(prediction_scaled)
    return prediction 




In [58]:
# test a single prediction 
user_inputs = {
    "pressure": 1011, 
    "humidity": 50,
    "city_name": "perth"   
}
print(f"predicted temp is: {predict(model, encoder, X_scaler, y_scaler, user_inputs)[0][0].round(2)} celcius")

predicted temp is: 24.7 celcius


/Users/jonathanneo/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  FutureWarning,


In [59]:
# save model 
import joblib 
joblib.dump(model, "output/model.sav")
joblib.dump(y_scaler, "output/y_scaler.sav")
joblib.dump(X_scaler, "output/x_scaler.sav")
joblib.dump(encoder, "output/encoder.sav")

['output/encoder.sav']